## Import dependencies, library

In [1]:
import tvm
import tvm.relay as relay
from tvm.contrib import graph_executor
from tvm.contrib.download import download_testdata
import numpy as np
from PIL import Image
import onnx
import os

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


## Downloading and Loading the ONNX Model

In [2]:
model_path = 'mobilenetv2_sigmoid.onnx'
onnx_model = onnx.load(model_path)

# Seed numpy's RNG to get consistent results
np.random.seed(0)

In [3]:
# Print the input names
print("Input names:")
for input_tensor in onnx_model.graph.input:
    print(input_tensor.name)

Input names:
args_0


## Downloading, Preprocessing, and Loading the Test Image

In [4]:
img_path = 'test.jpg'

# Resize it to 224x224
resized_image = Image.open(img_path).resize((224, 224))
img_data = np.asarray(resized_image).astype("float32")

# Normalize according to the ImageNet input specification
norm_img_data = img_data/255.0

# Add the batch dimension, as we are expecting 4-dimensional input: NCHW.
img_data = np.expand_dims(norm_img_data, axis=0)

## Define Compile Target

choose cpu target: llvm

In [5]:
target = "llvm"

## Compile the Model With Relay

In [6]:
input_name = input_tensor.name
shape_dict = {input_name: img_data.shape}

mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


## Run AI module

In [7]:
dtype = "float32"
module.set_input(input_name, img_data)
module.run()

## Get output from module

In [8]:
tvm_output = module.get_output(0).numpy()

## Benchmark Module Performance

In [9]:
unoptimized = module.benchmark(dev, number=1, repeat=30)
print(unoptimized)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  13.4164      12.7265      25.9232      12.1900       2.7094   
               


## Postprocess the output

In [10]:
from scipy.special import softmax

# # Download a list of labels
# labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
# labels_path = download_testdata(labels_url, "synset.txt", module="data")

# with open(labels_path, "r") as f:
#     labels = [l.rstrip() for l in f]

labels = ["cracked", "normal"]

# Open the output and read the output tensor
scores = softmax(tvm_output)
scores = np.squeeze(scores)
ranks = np.argsort(scores)[::-1]
for rank in ranks[0:5]:
    print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='plastic' with probability=0.398203
class='glass' with probability=0.152039
class='paper' with probability=0.150148
class='metal' with probability=0.149947
class='cardboard' with probability=0.149663


## Import model tuning dependencies

In [11]:
# import tvm.auto_scheduler as auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import autotvm

## Set LocalRunner

Set up some basic parameters for the runner. The runner takes compiled code
that is generated with a specific set of parameters and measures the
performance of it. ``number`` specifies the number of different
configurations that we will test, while ``repeat`` specifies how many
measurements we will take of each configuration. ``min_repeat_ms`` is a value
that specifies how long need to run configuration test. If the number of
repeats falls under this time, it will be increased. This option is necessary
for accurate tuning on GPUs, and is not required for CPU tuning. Setting this
value to 0 disables it. The ``timeout`` places an upper limit on how long to
run training code for each tested configuration.



In [12]:
number = 8
repeat = 1
min_repeat_ms = 0  # since we're tuning on a CPU, can be set to 0
timeout = 10  # in seconds

# create a TVM runner
runner = autotvm.LocalRunner(
    number=number,
    repeat=repeat,
    timeout=timeout,
    min_repeat_ms=min_repeat_ms,
    enable_cpu_cache_flush=True,
)

## Set tuning options

Create a simple structure for holding tuning options. We use an XGBoost
algorithim for guiding the search. For a production job, you will want to set
the number of trials to be larger than the value of 20 used here. For CPU we
recommend 1500, for GPU 3000-4000. The number of trials required can depend
on the particular model and processor, so it's worth spending some time
evaluating performance across a range of values to find the best balance
between tuning time and model optimization. Because running tuning is time
intensive we set number of trials to 10, but do not recommend a value this
small. The ``early_stopping`` parameter is the minimum number of trails to
run before a condition that stops the search early can be applied. The
measure option indicates where trial code will be built, and where it will be
run. In this case, we're using the ``LocalRunner`` we just created and a
``LocalBuilder``. The ``tuning_records`` option specifies a file to write
the tuning data to.



## Tuning Options

Create a simple structure for holding tuning options.



In [16]:
tuning_option = {
    "tuning_records": "mobilenetv2.json",
    "use_transfer_learning": True,
    "tuner": "xgb",
    "n_trial": 1000,
    "early_stopping": 100,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(build_func="default"), runner=runner
    ),
}

## Extract task from model

In [17]:
tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

## Tune task sequentially

In [18]:
def tune_tasks(
    tasks,
    measure_option,
    tuner="xgb",
    n_trial=1000,
    early_stopping=100,
    tuning_records="tuning.json",
    use_transfer_learning=True,
):
    # create tmp log file
    tmp_log_file = tuning_records + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(tsk, loss_type="rank")
        elif tuner == "xgb_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="knob")
        elif tuner == "xgb_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="itervar")
        elif tuner == "xgb_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="curve")
        elif tuner == "ga":
            tuner_obj = GATuner(tsk, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(tsk)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        # process tuning
        tsk_trial = n_trial
        tuner_obj.tune(
            n_trial=tsk_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(tsk_trial, prefix=prefix),
                autotvm.callback.log_to_file(tmp_log_file),
            ],
        )

    # pick best records to a cache file
    autotvm.record.pick_best(tmp_log_file, tuning_records)
    os.remove(tmp_log_file)

In [19]:
tune_tasks(tasks, **tuning_option)

[Task  1/34]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/1500) | 0.00 s

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__

[Task  1/34]  Current/Best:    2.10/   7.39 GFLOPS | Progress: (30/1500) | 14.84 s Done.
[Task  2/34]  Current/Best:    6.69/  10.04 GFLOPS | Progress: (20/1500) | 6.68 s Done.


/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__

[Task  3/34]  Current/Best:   19.56/  19.71 GFLOPS | Progress: (56/1500) | 50.96 s

...

[Task 14/34]  Current/Best:   69.74/ 109.59 GFLOPS | Progress: (56/1500) | 14.18 s

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__

[Task 14/34]  Current/Best:   12.60/ 109.59 GFLOPS | Progress: (120/1500) | 42.83 s

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__

[Task 14/34]  Current/Best:   47.57/ 116.51 GFLOPS | Progress: (184/1500) | 76.61 s

## Compiling an Optimized Model with Tuning Data

As an output of the tuning process above, we obtained the tuning records
stored in ``resnet-18-v2-autotuning.json``.

Now that tuning data for the model has been collected, we can re-compile the
model using optimized operators to speed up our computations.



In [ ]:
with autotvm.apply_history_best(tuning_option["tuning_records"]):
    with tvm.transform.PassContext(opt_level=3, config={}):
        lib = relay.build(mod, target=target, params=params)

In [ ]:
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

In [ ]:
dtype = "float32"
module.set_input(input_name, img_data)
module.run()
tvm_output = module.get_output(0).numpy()

scores = softmax(tvm_output)
scores = np.squeeze(scores)
ranks = np.argsort(scores)[::-1]
for rank in ranks[0:5]:
    print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123159 tiger cat' with probability=0.464309
class='n02123045 tabby, tabby cat' with probability=0.328435
class='n02124075 Egyptian cat' with probability=0.154237
class='n02127052 lynx, catamount' with probability=0.034955
class='n02128385 leopard, Panthera pardus' with probability=0.003649


## Comparing the Tuned and Untuned Models
We want to collect some basic performance data associated with this optimized
model to compare it to the unoptimized model. Depending on your underlying
hardware, number of iterations, and other factors, you should see a performance
improvement in comparing the optimized model to the unoptimized model.



In [ ]:
optimized = module.benchmark(dev, number=1, repeat=30)

print("Unoptimized: %s" % (unoptimized))
print("Optimized: %s" % (optimized))

Unoptimized: Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  53.4859      53.4593      53.7874      53.3423       0.1066   
               
Optimized: Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  42.7042      42.6927      43.0606      42.4750       0.1293   
               
